In [1]:
import os
import sys
import json

curdir = os.getcwd()
while 'config.json' not in os.listdir(curdir):
    curdir = os.path.dirname(curdir)
with open(os.path.join(curdir,'config.json'), 'r') as f:
    dataset = json.load(f)['dataset']

curdir = os.getcwd()
while 'filepathhelper.py' not in os.listdir(curdir):
       curdir = os.path.dirname(curdir)
sys.path.append(curdir)
import filepathhelper

In [2]:
import pickle
import os
import json
import pandas as pd
import numpy as np
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

In [3]:
winissues = pd.read_csv(filepathhelper.path(dataset,'winissue.csv'))
assignees = pd.read_csv(filepathhelper.path(dataset,'assignee.csv'),sep=';')
# assignees = assignees[(assignees['issuekey'].isin(winissues['issuekey']))]
teams = pd.read_csv(filepathhelper.path(dataset,'team.csv'))
# teams = teams[(teams['issuekey'].isin(winissues['issuekey']))]
teams = teams[(teams['issuekey'].isin(assignees['issuekey']))]
assignees.set_index('issuekey',inplace=True)

# training dataset

In [4]:
issue_team = {}
for key,issue in tqdm(teams.iterrows(),total=teams.shape[0]):
#     print(str(issue['dev'])=='nan')
    issuekey = issue['issuekey']
    if issuekey not in issue_team:
        issue_team[issuekey]={'developer':set(),'integrator':set(),'tester':set(),'reviewer':set(),'assignee':set()}
    team = issue_team[issuekey]
    if str(issue['dev'])!='nan':
        team['developer'].add(issue['dev'])
    if str(issue['integrator'])!='nan':
        team['integrator'].add(issue['integrator'])
    if str(issue['peer'])!='nan':
        team['reviewer'].add(issue['peer'])
    if str(issue['tester'])!='nan':
        team['tester'].add(issue['tester'])
    asn = assignees.loc[issuekey]
    if asn is not None:
        nam = asn['username']
        if str(nam)!='nan':
            team['assignee'].add(nam)

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [5]:
for issuekey in tqdm(issue_team):
    issue = issue_team[issuekey]
    for role in issue:
        issue[role] = list(issue[role])

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [6]:
actual = []
for issuekey in tqdm(issue_team):
    issue= {'issue':issuekey,'r':[{'rank':-1,'team':issue_team[issuekey]}]}
    actual.append(issue)

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [7]:
with open(filepathhelper.path(dataset,'actual_team.json'), 'w') as outfile:
    json.dump(actual, outfile)

# Train File

In [7]:
trainissuekey = pd.read_csv(filepathhelper.path(dataset,'trainissuekey.csv'))
descript = pd.read_csv(filepathhelper.path(dataset,'description.csv'))
teams = teams[(teams['issuekey'].isin(trainissuekey['issuekey']))]

In [8]:
with open(filepathhelper.path(dataset,'ItoC'),'rb') as f:
    ItoC = pickle.load(f)

In [9]:
issue_team = {}
for key,issue in tqdm(teams.iterrows(),total=teams.shape[0]):
#     print(str(issue['dev'])=='nan')
    issuekey = issue['issuekey']
    if issuekey not in issue_team:
        issue_team[issuekey]={'developer':set(),'integrator':set(),'tester':set(),'reviewer':set(),'assignee':set()}
    team = issue_team[issuekey]
    if str(issue['dev'])!='nan':
        team['developer'].add(issue['dev'])
    if str(issue['integrator'])!='nan':
        team['integrator'].add(issue['integrator'])
    if str(issue['peer'])!='nan':
        team['reviewer'].add(issue['peer'])
    if str(issue['tester'])!='nan':
        team['tester'].add(issue['tester'])
    asn = assignees.loc[issuekey]
    if asn is not None:
        nam = asn['username']
        if str(nam)!='nan':
            team['assignee'].add(nam)

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [10]:
for issuekey in tqdm(issue_team):
    issue = issue_team[issuekey]
    for role in issue:
        issue[role] = list(issue[role])

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [12]:
actual = []
for issuekey in tqdm(issue_team):
    context = descript[(descript['issuekey']==issuekey)].iloc[0]['description']
    if issuekey in ItoC:
        components = list(ItoC[issuekey])
    else:
        components = []
    issue= {'issue':issuekey,'context':context,'components':components,'r':[{'rank':-1,'team':issue_team[issuekey]}]}
    actual.append(issue)

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [13]:
with open(filepathhelper.path(dataset,'train_team.json'), 'w') as outfile:
    json.dump(actual, outfile)

# train team for hit and not hit

In [11]:
import hashlib

def hash_team(team):
    t_sorted = {i:sorted(team[i]) if type(team[i]) == list else team[i] for i in team}
    ## hash team
    t_hash = hashlib.sha1(json.dumps(t_sorted, sort_keys=True).encode()).hexdigest()
    return t_hash
with open(filepathhelper.path(dataset,'train_team_back.json')) as json_file:
    data = json.load(json_file)
    issueteam = pd.DataFrame(data)
    
hashteam = set()
for key,issue in tqdm(issueteam.iterrows(),total=issueteam.shape[0]):
    team = issue['r'][0]['team']
    hashteam.add(hash_team(team))

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


In [12]:
import random
random.seed(123)
with open(filepathhelper.path(dataset,'rp'),'rb') as f:
    rp = pickle.load(f)
def randnew(role,nummember):
    newmember = []
    for i in range(nummember):
#         nm = random.choice(rp[role])
#         while nm in hituser:
#             nm = random.choice(rp[role])
#         newmember.append(nm)
        newmember.append(random.choice(rp[role]))
    return newmember
def rolenewteam(team):
    for i in rp:
        team[i] = randnew(i,len(team[i]))
    return team

rp['developer'] =rp['dev']
rp['tester'] = rp['test']
rp['reviewer'] = rp['peer']
rp.pop('dev')
rp.pop('test')
rp.pop('peer')
assignees = pd.read_csv(filepathhelper.path(dataset,'assignee.csv'),sep=';')
assignees = assignees[(assignees['issuekey'].isin(trainissuekey['issuekey']))]
rp['assignee'] = list(set(assignees['username']))
rp.keys()

dict_keys(['integrator', 'developer', 'tester', 'reviewer', 'assignee'])

In [13]:
actual2=[]
for issuekey in tqdm(issue_team):
    if issuekey in ItoC:
        components = list(ItoC[issuekey])
    else:
        components = []
    count = 0
    cur_team = issue_team[issuekey]
    cur_hash = hash_team(cur_team)
    actual2.append({'issue':'real_'+issuekey,'components':components,'r':[{'rank':-1,'team':cur_team}]})
    while count<100:
        newteam = rolenewteam(cur_team.copy())
        if hash_team(newteam) != cur_hash:
            actual2.append({'issue':'gen'+str(count)+'_'+issuekey,'components':components,'r':[{'rank':-1,'team':newteam}]})
            count=count+1
with open(filepathhelper.path(dataset,'train_team.json'), 'w') as outfile:
    json.dump(actual2, outfile)  

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
